In [70]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from datetime import datetime

In [71]:
test = pd.read_csv("test.csv") #Dataframe untuk Test machine learning kita
train = pd.read_csv("train.csv") #Dataframe yg akan kita latih

In [72]:
show = pd.concat([train.head(4), train.tail(4)])
show

,ID,date,cluster_id,electricity_consumption,temperature_2m_max,temperature_2m_min,apparent_temperature_max,apparent_temperature_min,sunshine_duration,daylight_duration,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration
0,cluster_1_2014-01-01,2014-01-01,cluster_1,358.032,10.8,4.2,5.5,0.4,53.003333,29787.533333,40.6,79.2,186.000000,1.176667,0.483333
1,cluster_2_2014-01-01,2014-01-01,cluster_2,548.247,12.2,4.3,8.6,-0.4,8195.656667,30650.350000,36.0,81.7,170.591118,3.383333,0.736667
2,cluster_3_2014-01-01,2014-01-01,cluster_3,758.303,12.9,-0.8,10.1,-4.9,16305.260000,31547.686667,20.9,44.3,159.467752,3.880000,0.716667
3,cluster_4_2014-01-01,2014-01-01,cluster_4,1072.077,10.8,4.7,6.7,0.6,9224.803333,30769.220000,34.7,82.8,184.339753,3.153333,0.676667
11684,cluster_1_2021-12-31,2021-12-31,cluster_1,354.565,14.7,7.7,13.4,5.2,19148.153333,29727.200000,20.7,37.4,201.332279,3.510000,0.513333
11685,cluster_2_2021-12-31,2021-12-31,cluster_2,507.510,16.2,7.0,16.3,4.5,20570.920000,30594.280000,23.3,40.7,186.250597,5.006667,0.733333
11686,cluster_3_2021-12-31,2021-12-31,cluster_3,705.999,15.9,5.9,15.9,3.2,26860.350000,31496.173333,15.1,27.7,196.223902,5.783333,0.720000
11687,cluster_4_2021-12-31,2021-12-31,cluster_4,992.349,14.6,5.5,13.3,3.3,23063.000000,30713.893333,16.4,28.8,192.980343,4.816667,0.560000


# 4 Cluster dengan rentang waktu 2014-2021.
#### Membuat model prediksi konsumsi listrik harian (dalam satuan GWh) 

- electricity_consumption     =  konsumsi listrik harian (GWh)
- temperature_2m_max          =  suhu maksimum 2 meter
- temperature_2m_min          =  suhu minimum 2 meter
- apparent_temperature_max    =  suhu terasa maksimum
- apparent_temperature_min    =  suhu terasa minimum
- sunshine_duration           =  durasi sinar matahari
- daylight_duration           =  durasi siang hari
- wind_speed_10m_max          =  kecepatan angin maksimum di 10m
- wind_gusts_10m_max          =  hembusan angin maksimum di 10m
- wind_direction_10m_dominant =  arah angin dominan di 10m
- shortwave_radiation_sum     =  total radiasi gelombang pendek
- et0_fao_evapotranspiration  =  evapotranspirasi referensi FAO

In [73]:
#cek data duplikat
train.duplicated().sum()

np.int64(0)

In [74]:
#cek data missing values
train.isnull().sum().sum()

np.int64(0)

In [75]:
#cek tipe data
train.dtypes

ID                              object
date                            object
cluster_id                      object
electricity_consumption        float64
temperature_2m_max             float64
temperature_2m_min             float64
apparent_temperature_max       float64
apparent_temperature_min       float64
sunshine_duration              float64
daylight_duration              float64
wind_speed_10m_max             float64
wind_gusts_10m_max             float64
wind_direction_10m_dominant    float64
shortwave_radiation_sum        float64
et0_fao_evapotranspiration     float64
dtype: object

In [76]:
#mencari hubungan tanggal dengan electricity_consumption yang memiliki outlier untuk menjawab nomor 5b
Q1 = train['electricity_consumption'].quantile(0.25)
Q3 = train['electricity_consumption'].quantile(0.75)
IQR = Q3 - Q1

# Ambang batas outlier
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Data yang termasuk outlier
outlier_data = train[(train['electricity_consumption'] < lower_bound) | (train['electricity_consumption'] > upper_bound)]

outlier_data['date'] = pd.to_datetime(outlier_data['date'])
outlier_data['year'] = outlier_data['date'].dt.year
outlier_data['month'] = outlier_data['date'].dt.month

outlier_grouped = outlier_data.groupby(['year', 'month']).size().reset_index(name='outlier_count')

outlier_grouped

C:\Users\User\AppData\Local\Temp\ipykernel_7204\2021057916.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outlier_data['date'] = pd.to_datetime(outlier_data['date'])
C:\Users\User\AppData\Local\Temp\ipykernel_7204\2021057916.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outlier_data['year'] = outlier_data['date'].dt.year
C:\Users\User\AppData\Local\Temp\ipykernel_7204\2021057916.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

,year,month,outlier_count
0,2015,1,3
1,2015,2,4
2,2016,1,5
3,2017,1,16
4,2018,2,8
5,2018,3,2
6,2018,12,1
7,2019,1,5
8,2021,2,2


In [77]:
#mencari konsumsi listrik rata-rata dari tiap cluster saat suhu minimum di bawah persentil 25 dan durasi sinar matahari di bawah rata-rata untuk nomor 5a
q1 = train['temperature_2m_min'].quantile(0.25)
mean_sun = train['sunshine_duration'].mean()


filter_condition = (train['temperature_2m_min'] < q1) & (train['sunshine_duration'] < mean_sun)
filtered_data = train[filter_condition]


avg_energy_per_cluster = filtered_data.groupby('cluster_id')['electricity_consumption'].mean().reset_index(name='average_energy_25')
avg_energy_per_cluster


,cluster_id,average_energy_25
0,cluster_1,446.717842
1,cluster_2,656.699589
2,cluster_3,887.137849
3,cluster_4,1290.826150


In [78]:
#ubah cluster id ke int agar bisa masuk ke model
train['cluster_id'] = train['cluster_id'].str.replace('cluster_', '')
train['cluster_id'] = train['cluster_id'].astype(int)

if 'date' in train.columns:
    train['date'] = pd.to_datetime(train['date'])
    train['year'] = train['date'].dt.year
    train['month'] = train['date'].dt.month
    train['day'] = train['date'].dt.day
    df = train.drop('date', axis=1)

In [ ]:
#hitung rata-rata konsumsi listrik per cluster untuk menjawab no 4
avg_energy = train.groupby('cluster_id')['electricity_consumption'].mean().reset_index()
baseline = avg_energy.loc[avg_energy['cluster_id'] == 1, 'electricity_consumption'].values[0]
avg_energy['ratio_to_cluster1'] = avg_energy['electricity_consumption'] / baseline
avg_energy


,cluster_id,electricity_consumption,ratio_to_cluster1
0,1,340.734297,1.000000
1,2,503.367172,1.477301
2,3,706.762751,2.074234
3,4,1018.389092,2.988807


In [80]:
#mengecek korelasi antar variabel terhadap electricity_consumption. abaikan index 11 kebawah. untuk menjawab nomor 5b

train.columns = train.columns.str.strip()
start_col = 'temperature_2m_max'
start_idx = train.columns.get_loc(start_col)
target_columns = train.columns[start_idx:]
correlations = {
    col: train['electricity_consumption'].corr(train[col]) for col in target_columns
}

correlation_df = pd.DataFrame(list(correlations.items()), columns=['feature', 'correlation'])

correlation_df

,feature,correlation
0,temperature_2m_max,-0.307107
1,temperature_2m_min,-0.363282
2,apparent_temperature_max,-0.300234
3,apparent_temperature_min,-0.350391
4,sunshine_duration,-0.227085
5,daylight_duration,-0.336066
6,wind_speed_10m_max,0.039322
7,wind_gusts_10m_max,0.063458
8,wind_direction_10m_dominant,0.009720
9,shortwave_radiation_sum,-0.259866


In [81]:
train.head()

,ID,date,cluster_id,electricity_consumption,temperature_2m_max,temperature_2m_min,apparent_temperature_max,apparent_temperature_min,sunshine_duration,daylight_duration,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration,year,month,day
0,cluster_1_2014-01-01,2014-01-01,1,358.032,10.8,4.2,5.5,0.4,53.003333,29787.533333,40.6,79.2,186.000000,1.176667,0.483333,2014,1,1
1,cluster_2_2014-01-01,2014-01-01,2,548.247,12.2,4.3,8.6,-0.4,8195.656667,30650.350000,36.0,81.7,170.591118,3.383333,0.736667,2014,1,1
2,cluster_3_2014-01-01,2014-01-01,3,758.303,12.9,-0.8,10.1,-4.9,16305.260000,31547.686667,20.9,44.3,159.467752,3.880000,0.716667,2014,1,1
3,cluster_4_2014-01-01,2014-01-01,4,1072.077,10.8,4.7,6.7,0.6,9224.803333,30769.220000,34.7,82.8,184.339753,3.153333,0.676667,2014,1,1
4,cluster_1_2014-01-02,2014-01-02,1,386.908,10.7,7.0,6.6,3.3,22372.000000,29850.226667,34.3,75.2,210.333465,3.640000,0.780000,2014,1,2


In [82]:
train.describe()

,date,cluster_id,electricity_consumption,temperature_2m_max,temperature_2m_min,apparent_temperature_max,apparent_temperature_min,sunshine_duration,daylight_duration,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration,year,month,day
count,11688,11688.000000,11688.000000,11688.000000,11688.000000,11688.000000,11688.000000,11688.000000,11688.000000,11688.000000,11688.000000,11688.000000,11688.000000,11688.000000,11688.000000,11688.000000,11688.000000
mean,2017-12-31 12:00:00,2.500000,642.313328,18.733385,7.174786,17.215315,4.784608,29302.656241,44070.779592,23.273058,46.420115,202.335554,12.742257,2.428585,2017.500342,6.522930,15.729637
min,2014-01-01 00:00:00,1.000000,207.472000,-1.900000,-10.800000,-8.100000,-15.400000,0.000000,29447.550000,6.800000,13.300000,0.002855,0.333333,0.173333,2014.000000,1.000000,1.000000
25%,2016-01-01 00:00:00,1.750000,414.376000,12.800000,2.500000,10.000000,-1.100000,19104.170833,35150.501667,17.400000,35.300000,134.384708,5.600000,0.990000,2016.000000,4.000000,8.000000
50%,2017-12-31 12:00:00,2.500000,602.672000,18.200000,7.100000,16.400000,4.400000,29917.360000,44185.056667,22.000000,43.900000,219.000000,12.140000,2.156667,2017.500000,7.000000,16.000000
75%,2020-01-01 00:00:00,3.250000,836.034500,24.300000,12.100000,24.000000,10.800000,40697.756667,52997.815833,28.000000,54.700000,281.609519,18.924167,3.616667,2020.000000,10.000000,23.000000
max,2021-12-31 00:00:00,4.000000,1658.348000,40.900000,23.200000,42.600000,23.800000,53498.910000,58501.420000,63.900000,126.700000,360.000000,29.983333,8.133333,2021.000000,12.000000,31.000000
std,NaN,1.118082,287.673473,7.504161,5.960263,9.085303,7.260336,13600.113712,9283.839471,7.993389,15.226372,100.697741,7.588077,1.607845,2.291236,3.448851,8.800469


In [83]:
#Boxplot Cek Outliers (ADA)
#for i in train.select_dtypes(include='number').columns:
#    plt.figure(figsize=(6,4))
#    sns.boxplot(x=train[i])
#    plt.title(f'Boxplot of {i}')
#    plt.show()

In [84]:
#mark data yang memiliki outlier dan ganti data yang outlier jadi data Q3 atau Q1 berdasarkan roundingannya
cols_with_outliers = [
    "temperature_2m_max",
    "temperature_2m_min",
    "apparent_temperature_max",
    "apparent_temperature_min",
    "sunshine_duration",
    "wind_speed_10m_max"
]

for col in cols_with_outliers:
    Q1 = train[col].quantile(0.25)
    Q3 = train[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    train[col] = train[col].clip(lower, upper)

In [85]:
#Boxplot cek outliers (Sudah Bersih)
#for i in train.select_dtypes(include='number').columns:
#    plt.figure(figsize=(6,4))
#    sns.boxplot(x=train[i])
#    plt.title(f'Boxplot of {i}')
    #plt.show()

In [86]:
#apakah hari = weekend
from datetime import date

train['date'] = pd.to_datetime(train['date'])
train['isWeekend'] = (train['date'].dt.dayofweek >= 5).astype(int)


def isWeekend(date):
  return date.weekday() >=5


def season(d):
  year = d.year
  spring_start = date(year, 3, 20)
  summer_start = date(year, 6, 21)
  autumn_start = date(year, 9, 23)
  winter_start = date(year, 12, 21)

  d_date = d.date()

  if spring_start <= d_date < summer_start:
    return 2 # Spring
  elif summer_start <= d_date < autumn_start:
    return 3 # Summer
  elif autumn_start <= d_date < winter_start:
    return 4 # Autumn
  else:
    return 1 # Winter
  
train['season'] = train['date'].apply(season)
train = train.drop('date', axis=1)

In [87]:
#cek rata rata electricity_consumption per cluster untuk menjawab nomor 4
cluster_1 = train.groupby("cluster_id")['electricity_consumption'].mean()

In [88]:
train.head()

,ID,cluster_id,electricity_consumption,temperature_2m_max,temperature_2m_min,apparent_temperature_max,apparent_temperature_min,sunshine_duration,daylight_duration,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration,year,month,day,isWeekend,season
0,cluster_1_2014-01-01,1,358.032,10.8,4.2,5.5,0.4,53.003333,29787.533333,40.6,79.2,186.000000,1.176667,0.483333,2014,1,1,0,1
1,cluster_2_2014-01-01,2,548.247,12.2,4.3,8.6,-0.4,8195.656667,30650.350000,36.0,81.7,170.591118,3.383333,0.736667,2014,1,1,0,1
2,cluster_3_2014-01-01,3,758.303,12.9,-0.8,10.1,-4.9,16305.260000,31547.686667,20.9,44.3,159.467752,3.880000,0.716667,2014,1,1,0,1
3,cluster_4_2014-01-01,4,1072.077,10.8,4.7,6.7,0.6,9224.803333,30769.220000,34.7,82.8,184.339753,3.153333,0.676667,2014,1,1,0,1
4,cluster_1_2014-01-02,1,386.908,10.7,7.0,6.6,3.3,22372.000000,29850.226667,34.3,75.2,210.333465,3.640000,0.780000,2014,1,2,0,1


In [89]:
# x_train = train.drop(columns=['electricity_consumption','ID'])
# y_train = train['electricity_consumption']

In [90]:
# x_train, x_val, y_train, y_val = train_test_split(
#     x_train, y_train, test_size=0.2, random_state=42
# )

In [91]:
# model = RandomForestRegressor(n_estimators=100, random_state=42)
# model.fit(x_train, y_train)

In [92]:
# y_pred = model.predict(x_val)
# rmse = np.sqrt(mean_squared_error(y_val, y_pred))

# print("RMSE di validation set:", rmse)

In [93]:
# import joblib

# cobaLAGI = 'cobaLAGI.joblib'
# joblib.dump(model, cobaLAGI)